# Working with Databases

### Takeaways for Today

1. Reading large data sets into memory can be surprisingly slow, and in some cases completely impractical. 
2. Databases offer a convenient way to perform selective *queries* on our data, allowing us to retrieve only the data that we want at any given time. 
3. The `sqlite3` package enables us to work with databases using Python commands, and the `pandas` package makes it easy to read the results of database queries as data frames. 

## Intro 

So far, we've worked exclusively with data that could conveniently fit in memory. Generally speaking, data sets of under 20 megabytes or so can be loaded easily on most modern laptops, although subsequent analysis activities may be slow. Larger data sets can be noticeably slow to even load into memory. For example, you may  have noticed that so far, I've been using a small subset of the NOAA-GHCN data corresponding to the decade 2011-2020. There's much  more data, and I have the complete data set in a file (also uploaded on CCLE) in the same directory as these lecture notes on my personal machine. The complete set of data is not too big to fit in memory, but it's large enough that it's noticeably slow to even load into pandas. Let's take a look. 

952 ms ± 44.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Two seconds just to load the data set! Note: I'm not even *down*loading the data set -- this is just how long it takes to move the data from my computer's storage into RAM. 

We can get a pretty good estimate of how much RAM is required to store the data using a handy method of Pandas data frames: 

(1359937, 14)

152.313072

The data contains 1.35M rows and 14 columns. Altogether, these consume roughly 150 MB of RAM. This is an appreciable fraction of the RAM on most laptops. In some cases, we might need to work with even larger data sets that don't fit in RAM at all. 

## Introducing Databases

When dealing with large data sets like these, it's relatively rare that we absolutely *have* to operate on the entire data set. In most cases, we can work with parts of the data at a time. In the context of the temperature data, 

1. We might want temperature measurements only between years 1990-2020. 
2. We might want temperature measurements only for a certain set of countries - maybe in Asia, say. 
3. We might only want temperature measurements only in the month of March. 
4. We might want a random 1% of all the data. 

**Databases** provide us with a structured way to move subsets of data from storage into memory. Python has a module called `sqlite3` (already installed) which we can use to create, manipulate, and query databases. There's also a very handy `pandas` interface, enabling us to efficiently create `pandas` data frames containing exactly the data that we want. Let's get started. 

### Creating and Populating Databases

After importing the `sqlite3` module, the first thing we should do is connect to a database. In case the specified database does not exist, instantiating the connection will also create an empty database with the specified name.  

A quick check in our file directory reveals that we now have a file called `temps.db`. Great! 

There are many ways to add data to a database. Since we are already familiar with Pandas data frames, we'll make use of some extremely convenient functionality which allows us to directly write data frames to a database. 

But wait! Wasn't the whole point of this to avoid reading in an entire data frame? Indeed! Pandas supplies a nice approach to this using the familiar `pd.read_csv()` function. Supplying a value of `chunksize` will cause `read_csv()` to return not a data frame but an *iterator*, each of whose elements is a piece of the data with number of rows equal to `chunksize`. The data is read "on the fly" -- i.e. it's not actually read until we start querying the iterator. Here's an example: 

,ID,Year,VALUE1,VALUE2,VALUE3,VALUE4,VALUE5,VALUE6,VALUE7,VALUE8,VALUE9,VALUE10,VALUE11,VALUE12
0,ACW00011604,1961,-89.0,236.0,472.0,773.0,1128.0,1599.0,1570.0,1481.0,1413.0,1174.0,510.0,-39.0
1,ACW00011604,1962,113.0,85.0,-154.0,635.0,908.0,1381.0,1510.0,1393.0,1163.0,994.0,323.0,-126.0
2,ACW00011604,1963,-713.0,-553.0,-99.0,541.0,1224.0,1627.0,1620.0,1596.0,1332.0,940.0,566.0,-108.0
3,ACW00011604,1964,62.0,-85.0,55.0,738.0,1219.0,1442.0,1506.0,1557.0,1221.0,788.0,546.0,112.0
4,ACW00011604,1965,44.0,-105.0,38.0,590.0,987.0,1500.0,1487.0,1477.0,1377.0,974.0,31.0,-178.0


Great! Actually, there are a few cleaning steps that we'll make before incorporating this data into our database. If you're not sure what's going on here, you may want to check the recent lecture in which we cover `stack`.  

,ID,Year,Month,Temp
0,ACW00011604,1961,1,-0.89
1,ACW00011604,1961,2,2.36
2,ACW00011604,1961,3,4.72
3,ACW00011604,1961,4,7.73
4,ACW00011604,1961,5,11.28


Ok! We are finally ready to populate a *table* in our database. 

You can think of a table as a data frame-like object, represented in SQLite rather than in Python. The `df.to_sql()` method writes to a specified table in the database (the `conn` object from earlier). We need to specify `if_exists` to ensure that we add each piece to the table, rather than overwriting them each time. 

Let's similarly add a table for the metadata in our database. This is a pretty small data set so we don't need to worry about reading it in by chunks. 

In [12]:
url = "https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/station-metadata.csv"


Now we have a database containing two tables. Let's just check that this is indeed the case. 

[('temperatures',), ('stations',)]


Ok wait -- there's actually a lot that just happened here! 

First: the *cursor* is our primary way to interact with the database. The cursor `execute`s *SQL commands*. SQL is actually its own mini-programming language specifically designed for interacting with databases. We do need to learn a bit of SQL in order to work with databases, but I promise it won't be too bad. 

Now let's take a look at the actual command that we `execute`d.

```sql
SELECT name FROM sqlite_master WHERE type='table'
```

Let's break this down: 

- `SELECT name`: Show me the entries in the `name` column
- `FROM sqlite_master`: of the `sqlite_master` table
- `WHERE type='table'`: subject to the condition that the entry in the `type` column of `sqlite_master` is equal to ``table`` (i.e. don't include other kinds of objects)

Finally, `cursor.fetchall()` returns the a list containing all the items returned by the query, which we then print.

The special `sqlite_master` table contains information on all the objects in the database. We don't usually query it when we want to obtain data, but we can query it to learn about what kinds of tables we have, what their columns are, etc. 

Let's get more detailed information about the items in each table. For example, we can inspect the column names and data types in each. This is a good way to check that we actually populated our database correctly. 

CREATE TABLE "temperatures" (
"ID" TEXT,
  "Year" INTEGER,
  "Month" INTEGER,
  "Temp" REAL
)
CREATE TABLE "stations" (
"ID" TEXT,
  "LATITUDE" REAL,
  "LONGITUDE" REAL,
  "STNELEV" REAL,
  "NAME" TEXT
)


This looks pretty good! We have two tables, called `temperatures` and `stations`. The column names are what we would expect. Notice that `sql` has automatically inferred the data types, such as `REAL` and `TEXT`, from the input. Handy! 

#### Basic Queries

Now we're ready to perform some basic queries on the data tables themselves. 

What data do we have for the year 1990? In `pandas` we would do this like: 

```
temperatures["id"][temperatures["year"] == 1990]
```

The SQL syntax is very different, but contains all the same ideas. 

- `SELECT`, like the syntax `temperatures["id"]`, controls which column(s) will be returned. 
- `FROM` tells us which table to return columns from. 
- `WHERE` is like the Boolean index `[temperatures["year"] == 1990]`. Only rows in which this criterion is satisfied will be returned. 

**NOTE**: SQL commands are case-insensitive, but it's considered good practice to place SQL keywords in ALL CAPS while names of columns and other content goes in lowercase. 

[('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',)]

Note that the result is returned as a tuple. This isn't very useful when we are just querying a single column, but we can also get multiple columns, in which case the tuple representation makes a bit more sense: 

[('ACW00011604', 4.51),
 ('ACW00011604', 5.91),
 ('ACW00011604', 6.01),
 ('ACW00011604', 8.11),
 ('ACW00011604', 12.31),
 ('ACW00011604', 15.81),
 ('ACW00011604', 16.71),
 ('ACW00011604', 17.11),
 ('ACW00011604', 12.11),
 ('ACW00011604', 9.51)]

We can include multiple criteria in `WHERE` using the Boolean operators `AND` and `OR`. 

[('ACW00011604', 4.51),
 ('AE000041196', 17.78),
 ('AEM00041184', 18.3),
 ('AEM00041194', 18.87),
 ('AEM00041216', 18.91),
 ('AEM00041217', 18.57),
 ('AFM00040911', 4.4),
 ('AFM00040948', 0.16),
 ('AFXLT299877', -5.6),
 ('AG000060390', 10.74)]

Which stations are either far south or far north? 

[('AYM00089013',),
 ('AYM00089108',),
 ('AYM00089314',),
 ('AYM00089329',),
 ('AYM00089332',),
 ('AYM00089345',),
 ('AYM00089377',),
 ('AYM00089577',),
 ('AYM00089799',),
 ('AYM00089873',)]

## Relations

What if we wanted the temperature measurements taken at elevations higher than 1000m above sea-level? This involves information from both the `temperature` table (for the temperatures themselves) and the `stations` table (for elevation info). In order to perform this kind of query, we need to incorporate relational information into our query. This kind of task is very closely related to the `pd.merge` operation that we recently learned. 

We know from our previous interactions with this data set that the `id` column in the `stations` table corresponds to the `id` column in the `temperature` table. We'd like to incorporate this relationship in our database. From here, the syntax gets only slightly more complicated. Here's what we'll do: 

1. We'll give the `temperatures` and `stations` tables aliases `T` and `S` just to make the command more readable. We can also split the command onto multiple lines. 
2. We'll need to add a `LEFT JOIN` command which indicates which column of `T` corresponds to which column of `S`. This is similar to `pd.merge` from a few lectures ago. There are also `RIGHT JOIN`, `INNER JOIN` and `OUTER JOIN` operators that control how duplicates and mismatched rows are handled. The `ON T.id = S.id` portion indicates which columns are expected to correspond. 
3. As usual, we also need to specify which columns we want (with `SELECT`) and any filter criteria (with `WHERE`). 

For readability, it's a good idea to break this up into multiple lines. 


[('AYM00089013', 4, -41.08),
 ('AYM00089013', 5, -48.4),
 ('AYM00089013', 6, -50.7),
 ('AYM00089013', 7, -49.14),
 ('AYM00089013', 10, -43.66),
 ('AYM00089013', 11, -30.94),
 ('AYM00089013', 12, -20.55),
 ('AYM00089013', 1, -19.54),
 ('AYM00089013', 2, -31.39),
 ('AYM00089013', 3, -39.56)]

## Tables into Pandas

The cursor is very good for prototyping your SQL queries, but once you've gotten the hang of it, the easiest approach is to use another convenience function from Pandas: 

This command takes a long time to execute, and it may seem that we haven't gained much from the database approach. However, it's important to remember that we're not JUST reading a table directly: there's a `JOIN` going on under the hood here. `JOIN`-type operations, including `pd.merge`, can be very expensive computationally. 

You can actually obtain columns from both tables this way: 

It's good practice to close your database connection once you're done using it. 

Next time you need to pull some data from your database, just reopen the connection! 

#### *Flashforward -- Sometime in The Future*

## Aggregation

SQLite3 contains some limited aggregation capabilities. For example, suppose we'd like to compute the average temperature at each station in each month which was either far north or far south. We already know how to do this in Pandas using `df.groupby().aggregate()`. Again, the `SQL` syntax is very different, but contains all the same ideas. The main changes below are that we apply functions to the columns we want to modify, specify a name for the aggregate column, and specify one or more GROUP BY columns. 

,NAME,Month,Mean Temperature
0,ALERT,1,-32.1
1,ALERT,2,-33.4
2,ALERT,3,-32.8
3,ALERT,4,-24.6
4,ALERT,5,-11.5
...,...,...,...
259,VIKTORIYA_ISLAND,8,-0.5
260,VIKTORIYA_ISLAND,9,-2.9
261,VIKTORIYA_ISLAND,10,-10.2
262,VIKTORIYA_ISLAND,11,-16.3


### Takeaways for Today

1. Reading large data sets into memory can be surprisingly slow, and in some cases completely impractical. 
2. Databases offer a convenient way to perform selective *queries* on our data, allowing us to retrieve only the data that we want at any given time. 
3. The `sqlite3` package enables us to work with databases using Python commands, and the `pandas` package makes it easy to read the results of database queries as data frames. 